네이버쇼핑 크롤링 코드입니다. 


검색어와 검색하고자 하는 페이지를 입력하면 크롤링합니다.

In [1]:
import re, requests, csv
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os

In [2]:
#크롬드라이버가 있는곳의 경로를 chromedriver까지 써줍니다.
path = os.getcwd().replace('\\','/', 10)
driver_path = path + '/chromedriver_win32/chromedriver'

#### 스크롤 함수 

네이버 쇼핑은 스크롤 하기 전에는 상위 5개 항목만 인식합니다. 

따라서 스크롤 해주는 함수가 필요합니다.

In [3]:
#출처 : https://hello-bryan.tistory.com/194
def scroll_page(driver, scroll_pause_sec):
    """driver, 스크롤 후 정지 초를 넘겨주면 맨 아래까지 스크롤해주는 함수"""
    # 스크롤 높이 가져옴
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(scroll_pause_sec)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

#### 크롤러
드라이버 패스와 네이버 랭킹사용여부(기본값 False일 경우 리뷰개수 순)을 넘겨주면 검색어, 페이지를 입력받아 리뷰와 쇼핑몰 판매정보를 크롤링합니다. 
 <br>
 <br>
 _price_info, reviews_ 는 딕셔너리 형태이며
크롤러는 다음 튜플을 반환합니다.
 ```(price_info, reviews)```
 
__price_info__

|gift |	mall | name	| price | shipping | title
---|---|---|---|---|---|
1||||||
2||||||



__reviews__

|content |	date | id	| mall | name | option | title
---|---|---|---|---|---|---|
1|||||||
2|||||||


In [19]:
def naver_shopping_crawler(driver_path, naver_rank = False):

    base_url = 'https://shopping.naver.com/'

    print('검색어를 입력해주세요 : ')
    word = input()

    print('로딩중')

    driver = webdriver.Chrome(driver_path)
    driver.get(base_url)
    print('...')
    time.sleep(2)
    
    #검색어 검색하기
    elem = driver.find_element_by_xpath('//*[@id="autocompleteWrapper"]/input[1]')
    elem.send_keys(word)
    elem.send_keys(Keys.RETURN)
    print('...')
    time.sleep(2)
    
    #페이지 맨 아래로 스크롤
    scroll_page(driver, 1)
    
    #위의 탭
    tabs = driver.find_elements_by_xpath('//a[@class="subFilter_filter__3Y-uy"]')
    
    #가격비교 클릭
    #상품별로 검색할때 xpath가 다르므로 수정 필요
    compare_price = tabs[1]
    compare_price.send_keys(Keys.ENTER)
    # 전체,가격비교,네이버페이,백화점/홈쇼핑,쇼핑윈도,해외직구
    
    print('...')
    time.sleep(1)
    
    
    #find_elements_by_xpath('//a[@class="subFilter_sort__rQUtM"]') 이게 작동을 안함
    if naver_rank == False:
        sort_by = driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div/div[3]/div[1]/div[1]/div/div[1]/a[5]')
        sort_by.click()
     # 네이버 랭크 옵션이 false면 리뷰순으로 보기 클릭
    #상품별로 검색할때 xpath가 다르므로 수정 필요

    print('...')
    time.sleep(2)
        
    html_list = driver.page_source 
    soup_list = BeautifulSoup(html_list)#, 'html.parser')

    #페이지 정보 받아와서 상품별 url 리스트로 만듦
   
    url_list = soup_list.find_all('div', {'class':"basicList_title__3P9Q7"})
    print('...')
    time.sleep(3)
    print("1페이지  제품 수 : " +str(len(url_list)))

    #가격비교에 있는 상품 수
    price_compare_count = soup_list.find_all('span', {'class':'subFilter_num__2x0jq'})[1]
    price_compare_count = int(re.sub('<.*?>|,','', str(price_compare_count)))

    if price_compare_count % 40 != 0:
        price_compare_pages = (price_compare_count //40) +1
    else:
        price_compare_pages = (price_compare_count //40)


    print("최대 수집가능 페이지 수 : ", str(price_compare_pages),'\n','수집을 원하는 페이지 수를 입력해주세요')
    n_pages = int(input())
    print('1페이지의 일부만 하실 경우 위에서부터 가져올 제품 개수를 입력해주세요\n',
          '아닐경우 40을 입력해주세요')
    n_product = int(input())

    assert n_pages <= price_compare_pages, "페이지 수 초과"

    price_info = []
    ap_0 = price_info.append

    reviews = []
    ap_1 = reviews.append

    num1 = 0

    for page in range(1,1+n_pages):


        num1+=1 #페이지 카운트
        print('--------------------'+str(num1)+'페이지 입니다----------------------\n')
        num2 = 0 #제품 카운트

        for url in tqdm(url_list[:n_product]):
            url = url.find('a', {'class':"basicList_link__1MaTN"})
            url_i = re.search(r'href=[\'"]?([^\'" >]+)', str(url), re.I|re.S).group(0)[6:]
            driver.get(url_i)
            html_product_i = driver.page_source
            soup_i=BeautifulSoup(html_product_i)

            try:
                review_count = soup_i.find_all('a', {'href':'#section_review'})[0]
                review_count = int(re.sub(r'[^0-9]','', str(review_count)))

            except:
                review_count = 0
            #print(review_count)

            name = soup_i.find('div', {'class':'h_area'})
            name = re.sub('<.+?>','', str(name.find('h2')), re.I|re.S).strip()

            #print(name)

            #제품 카운트
            num2+=1
            print(str(num1)+'페이지 '+str(num2)+'번 제품입니다\n')

            # 판매자 이름
            
            try:
                price_list = soup_i.find_all('table', {'class':'tbl tbl_v'})
            except:
                print("이 상품은 품절입니다.")
                driver.back()
                continue

            for price_i in price_list:

                price_mall = price_i.find('a', {'class':'_priceListMallLogo'})
                price_mall = re.search('data-mall-name=\".*?\"', str(price_mall), re.I|re.S).group(0)[16:-1]

                price_title = price_i.find('a',{'class':'goods_tit'})
                price_title = re.sub("<.*?>", '', str(price_title))
                price_title = re.sub("\n", '', str(price_title)).strip()

                gift = price_i.find('p',{'class':'goods_gift'})
                gift = re.sub("<.*?>", '', str(gift))
                gift = re.sub("\n", '', str(gift)).strip()

                price = price_i.select('td > a > span')[-1]
                price = int(re.sub("<.*?>|,", '', str(price)))

                shipping = price_i.select('td > p')[1]
                shipping = re.sub("<.*?>|,", '', str(shipping))


                price_info_i = {'name':name,
                               'mall':price_mall,
                               'title':price_title,
                               'gift':gift,
                               'price':price,
                               'shipping':shipping}

                ap_0(price_info_i)
            
            #총 리뷰 페이지 수
            #리뷰는 한페이지에 20개까지 표시됩니다.
            if review_count == 0:
                print("이 제품은 리뷰가 없습니다. \n")
                driver.back()
                continue
            if review_count % 20 != 0:
                review_page = (review_count // 20) + 1
            else:
                review_page = (review_count // 20)


            for page in tqdm(range(1, review_page+1)):

                time.sleep(2)

                html_for_rvw = driver.page_source
                soup_for_rvw = BeautifulSoup(html_for_rvw, 'html.parser')
                
                
                #해당 페이지에 현재 보이는 리뷰들의 리스트
                review_list = soup_for_rvw.find('ul', {'id':'_review_list'})
                review_list = review_list.find_all('div', {'class':'atc_area'})

                for review in review_list:

                    rvw_info = review.find_all('span', {'class':'info_cell'})
                    mall = re.sub('<.+?>', '', str(rvw_info[0]), re.I|re.S)
                    id_ =  re.sub('<.+?>', '', str(rvw_info[1]), re.I|re.S)
                    date = re.sub('<.+?>', '', str(rvw_info[2]), re.I|re.S)
                    try: #상품 옵션은 있을 수도 없을 수도 있습니다.
                        option = re.sub('<.+?>', '', str(rvw_info[3]), re.I|re.S)
                    except:
                        option = ''

                    rvw_title = review.find('p', {'class':'subjcet'})
                    rvw_title = re.sub('<.+?>', '', str(rvw_title), re.I|re.S)

                    rvw_content = review.find('div', {'class':'atc'})
                    rvw_content = re.sub('<.+?>', '', str(rvw_content), re.I|re.S)
                    
                    #리뷰정보를 딕셔너리로 저장
                    rvw_dic = {'name':name,
                                'mall' :mall,
                               'id' : id_,
                               'date' : date,
                               'option' :option,
                               'title' :rvw_title,
                               'content' :rvw_content}
                    ap_1(rvw_dic)
                
                #현재리뷰페이지가 마지막이아니라면, 다음리뷰페이지 버튼을 클릭합니다.
                if page < review_page:
                    path = '//a[@onclick="shop.detail.ReviewHandler.page({}, \'_review_paging\'); return false;"]'.format(page+1)
                    button = driver.find_element_by_xpath(path)
                    button.send_keys(Keys.ENTER)

            driver.back()

        #현재 페이지가 지정한 페이지 수보다 적다면, 다음 페이지로 가요
        if page < n_pages:
            next_page_path = '//a[@class="pagination_next__1ITTf"]'
            driver.find_element_by_xpath(next_page_path).click()
            time.sleep(2)
            scroll_page(driver, 1)

    driver.close()
    
    return (price_info,reviews)

## 실행

In [14]:
#실행결과를 data변수에 저장
data = naver_shopping_crawler(driver_path, True)

검색어를 입력해주세요 : 
에르메스
로딩중
...
...
...
...
...
1페이지  제품 수 : 40
최대 수집가능 페이지 수 :  117 
 검색을 원하는 페이지 수를 입력해주세요
2
1페이지만 하실 경우 위에서부터 가져올 제품 개수를 입력해주세요
 아닐경우 40을 입력해주세요
40
--------------------1페이지 입니다----------------------



1페이지 1번 제품입니다




1페이지 2번 제품입니다




1페이지 3번 제품입니다




1페이지 4번 제품입니다



...


2페이지 35번 제품입니다




2페이지 36번 제품입니다




2페이지 37번 제품입니다




2페이지 38번 제품입니다

이 제품은 리뷰가 없습니다. 

2페이지 39번 제품입니다




2페이지 40번 제품입니다



In [18]:
naver_reviews = pd.DataFrame(data[1])
naver_reviews.to_csv("naver_reiviews")
naver_reviews.head()

,content,date,id,mall,name,option,title
0,"생각했던 것만큼 정말 예쁜 제품이네요 ㅎㅎ골드랑 고민했었는데, 에토프하길 잘 한듯합...",20.09.19.,love****,베스티 런던,해외에르메스 오켈리 로즈골드 자물쇠 펜던트 목걸이 H078852CD18 H07885...,컬러: 에토프,"생각했던 것만큼 정말 예쁜 제품이네요 ㅎㅎ골드랑 고민했었는데, 에토프하길 잘 한듯합..."
1,안전하고 빠른배송 감사해요,20.09.25.,woal****,베스티 런던,해외에르메스 오켈리 로즈골드 자물쇠 펜던트 목걸이 H078852CD18 H07885...,컬러: 골드,안전하고 빠른배송 감사해요
2,목걸이가 자꾸 뒤집어지긴해도 너무이뻐요!!!로골이라 더이뿜,20.09.09.,jugn****,베스티 런던,해외에르메스 오켈리 로즈골드 자물쇠 펜던트 목걸이 H078852CD18 H07885...,컬러: 골드,목걸이가 자꾸 뒤집어지긴해도 너무이뻐요!!!로골이라 더이뿜
3,빨리 와서 좋아요 꼼꼼히 포장해주셨고 이쁩니다 쇼핑백도 넣어주셨음 더 좋았을텐데 조...,20.09.01.,hy_j****,베스티 런던,해외에르메스 오켈리 로즈골드 자물쇠 펜던트 목걸이 H078852CD18 H07885...,컬러: 에토프,빨리 와서 좋아요 꼼꼼히 포장해주셨고 이쁩니다 쇼핑백도 넣어주셨음 더 좋았을텐데 조...
4,디자인.이쁘고.착용감도.좋아요!굿입니다!,20.09.09.,qkrt****,베스티 런던,해외에르메스 오켈리 로즈골드 자물쇠 펜던트 목걸이 H078852CD18 H07885...,컬러: 에토프,디자인.이쁘고.착용감도.좋아요!굿입니다!


In [17]:
naver_price_info  = pd.DataFrame(data[0])
naver_price_info.to_csv("naver_price_info.csv")
naver_price_info.head()

,gift,mall,name,price,shipping,title
0,,베스티 런던,해외에르메스 오켈리 로즈골드 자물쇠 펜던트 목걸이 H078852CD18 H07885...,345000,25000원,에르메스 오켈리 로즈골드 자물쇠 펜던트 목걸이 스몰 H078852CD18/H0788...
1,,베스티 런던,해외에르메스 오켈리 로즈골드 자물쇠 펜던트 목걸이 H078852CD18 H07885...,409000,10000원,[에르메스] 오켈리 로즈골드 자물쇠 펜던트 목걸이 스몰 H078852CD18/H07...
2,,런던쇼룸,해외에르메스 목걸이 액세서리 여성 1B 제품 H147991FP,531050,무료배송,[Hermes] 에르메스 팝 H 팬던트 네크리스 실버 (4컬러) H147991FP
3,,페퍼센트,해외에르메스 목걸이 액세서리 여성 1B 제품 H147991FP,501000,10000원,에르메스 H 펜던트 JAUNE MIMOSA H147991FP1R
4,,더모어런던,해외에르메스 목걸이 액세서리 여성 1B 제품 H147991FP,520200,10000원,[HERMES] Pop H pendant (Jaune Mimosa) 에르메스 팝에이...


#### 수정 사항
+ 크롤링 원하는 상품 +1이 되는 문제 (수정 완료)


+ compare_price 부분 수정 

    + //*[@id="__next"]/div/div[2]/div[2]/div[4]/div[1]/div[1]/ul/li[2]/a
    + 검색어마다 구조가 다름
    
    
+ url_list도 저기 걸리는 태그가 검색어마다 다름


+ 스크롤 내리는거 수정(수정완료)


+ 로딩시간 기다리기 표시